# Projeto 1 - Ciência dos Dados

Nome: Esther Dagir

Nome: Lila Hadba

**Atenção:** Serão permitidos grupos de três pessoas, mas com uma rubrica mais exigente. Grupos deste tamanho precisarão fazer um questionário de avaliação de trabalho em equipe

___
Carregando algumas bibliotecas:

In [28]:
%matplotlib inline
import pandas as pd
from math import*
import matplotlib.pyplot as plt
import numpy as np
import os
from IPython.display import display
#!pip install emoji

from emoji import UNICODE_EMOJI
pd.options.display.max_rows = 13

ModuleNotFoundError: No module named 'emoji'

**Em `filename`, coloque o nome do seu arquivo de dados!**

In [29]:
import os

filename = 'Netflix.xlsx'
if filename in os.listdir():
    print(f'Encontrei o arquivo {filename}, tudo certo para prosseguir com a prova!')
else:
    print(f'Não encontrei o arquivo {filename} aqui no diretório {os.getcwd()}, será que você não baixou o arquivo?')

Encontrei o arquivo Netflix.xlsx, tudo certo para prosseguir com a prova!


Carregando a base de dados com os tweets classificados como relevantes e não relevantes:

In [30]:
train = pd.read_excel(filename)
train.head(5)

,Treinamento,Categoria
0,saiu da netflix? q sdd de full house! amo muit...,1
1,rt @karolcaipira: gente vcs tem que entender u...,1
2,rt @karolcaipira: gente vcs tem que entender u...,1
3,"mano, não tem mais nada p assistir na netflix ...",1
4,"@jornaloglobo vcs assistiram o filme, primeira...",1


In [31]:
test = pd.read_excel(filename, sheet_name = 'Teste')
test.head(5)

,Teste,Categoria
0,rt @blogdadireita: min. damares alves quer pro...,0
1,@rachwlgrwn no off eu fui até a terceira e se ...,1
2,eu quero julie e os fantasmas na netflix pra c...,1
3,nunca vou perdoar a netflix https://t.co/ssrbk...,1
4,seria demais querer que a @damaresalves e os e...,0


___
## Classificador automático de sentimento


Faça aqui uma descrição do seu produto e o que considerou como relevante ou não relevante na classificação dos tweets.



Nosso produto é um serviço de streaming de filmes e séries. De uma escala de 0 a 2 temos que:
0 é "nada relevante" - abrange tweets que não influenciam no desempenho da plataforma
1 é "relevante" - tweets que são recomendações ou reinvidicações para a plataforma
2 é "muito relevante" - todos os tweets que estão relacionados ao gosto positivamente e negativamente à plataforma

In [32]:
#juntar num textao
dfnaorelevante = train.loc[train['Categoria'] == 0]
textao0 = " ".join(dfnaorelevante.Treinamento)
dfnaorelevante

,Treinamento,Categoria
5,rt @bunnyfeveer: esses dias estreou o filme mi...,0
8,damares quer proibir filme da netflix acusado ...,0
9,@elgwellm meu pai tá me pirraçando e até agora...,0
10,@pacefeco o que tem a ver a globo com a netfli...,0
17,eu começo a ver umas coisa tão aleatória não n...,0
...,...,...
290,nada tira da minha cabeça que ela já sabia que...,0
294,os militantes que cancelaram a netflix pelo fi...,0
295,rt @craibeibe: inveja de você? kkkkkkkkkkkkkkk...,0
296,quero saber quando é q vou poder colocar as pi...,0


In [33]:
dfnaorelevante = train.loc[train['Categoria'] == 0]
dfnaorelevante

,Treinamento,Categoria
5,rt @bunnyfeveer: esses dias estreou o filme mi...,0
8,damares quer proibir filme da netflix acusado ...,0
9,@elgwellm meu pai tá me pirraçando e até agora...,0
10,@pacefeco o que tem a ver a globo com a netfli...,0
17,eu começo a ver umas coisa tão aleatória não n...,0
...,...,...
290,nada tira da minha cabeça que ela já sabia que...,0
294,os militantes que cancelaram a netflix pelo fi...,0
295,rt @craibeibe: inveja de você? kkkkkkkkkkkkkkk...,0
296,quero saber quando é q vou poder colocar as pi...,0


In [34]:
#juntar num textao
dfrelevante = train.loc[train['Categoria'] == 1]
dfrelevante
textao1 = " ".join(dfrelevante.Treinamento)

In [35]:
dfrelevante = train.loc[train['Categoria'] == 1]
dfrelevante

,Treinamento,Categoria
0,saiu da netflix? q sdd de full house! amo muit...,1
1,rt @karolcaipira: gente vcs tem que entender u...,1
2,rt @karolcaipira: gente vcs tem que entender u...,1
3,"mano, não tem mais nada p assistir na netflix ...",1
4,"@jornaloglobo vcs assistiram o filme, primeira...",1
...,...,...
291,rt @awaecharts: anne with an e na netflix chil...,1
292,rt @baptistafairy: eu tô muito sedenta pela se...,1
293,netflix já foi melhor .,1
297,"@netflixbrasil netflix querida, vê se consegue...",1


In [36]:
#juntar num textao
'''dfmuitorelevante = train.loc[train['Categoria'] == 2]
dfmuitorelevante
textao2 = " ".join(dfmuitorelevante.Treinamento)'''

'dfmuitorelevante = train.loc[train[\'Categoria\'] == 2]\ndfmuitorelevante\ntextao2 = " ".join(dfmuitorelevante.Treinamento)'

In [37]:
'''dfmuitorelevante = train.loc[train['Categoria'] == 2]
dfmuitorelevante'''

"dfmuitorelevante = train.loc[train['Categoria'] == 2]\ndfmuitorelevante"

In [38]:
test['Categoria'].value_counts(True)

1    0.563786
0    0.436214
Name: Categoria, dtype: float64

___
### Montando um Classificador Naive-Bayes

Considerando apenas as mensagens da planilha Treinamento, ensine  seu classificador.
Antes, foi realizado uma função para filtrar  os textos, trocando certos caracteres por espaço e etc.

In [39]:
import re 


def cleanup(text):
    """
        Função de limpeza muito simples que troca alguns sinais básicos por espaços
    """
    #import string
    punctuation = '[!-.:?;]' # Note que os sinais [] são delimitadores de um conjunto.
    pattern = re.compile(punctuation)
    text_subbed = re.sub(pattern, '', text)
    text_subbed = re.sub('http', '', text_subbed)
    return text_subbed

def is_emoji(a):
    return a in UNICODE_EMOJI

#### Ensinando o classificador

In [40]:
netflix0limpo = dfnaorelevante['Treinamento'].apply(cleanup)

print(netflix0limpo)

5      rt @bunnyfeveer esses dias estreou o filme mig...
8      damares quer proibir filme da netflix acusado ...
9      @elgwellm meu pai tá me pirraçando e até agora...
10     @pacefeco o que tem a ver a globo com a netfli...
17     eu começo a ver umas coisa tão aleatória não n...
                             ...                        
290    nada tira da minha cabeça que ela já sabia que...
294    os militantes que cancelaram a netflix pelo fi...
295    rt @craibeibe inveja de você kkkkkkkkkkkkkkkkk...
296    quero saber quando é q vou poder colocar as pi...
299    @boatosorg bolsonaristas já estão importando a...
Name: Treinamento, Length: 145, dtype: object


In [41]:
lista_naorelevantes = []
palavrasnaorelevantes = []
for tt in netflix0limpo:
    lista_naorelevantes.append(tt.split())
    
for a in lista_naorelevantes:
    for b in a:
        palavrasnaorelevantes.append(b)

In [42]:
serie_0 = pd.Series(palavrasnaorelevantes)
serie_0

0                      rt
1            @bunnyfeveer
2                   esses
3                    dias
4                 estreou
              ...        
3036           importando
3037                    a
3038                 fake
3039               abaixo
3040    s//tco/tnrpdhzlgj
Length: 3041, dtype: object

In [43]:
tabela_naorelevante = serie_0.value_counts()
tabela_naorelevante

netflix        129
a              104
de              84
que             76
o               59
              ... 
resto            1
pois             1
igualzinho       1
história         1
@sapphicute      1
Length: 1266, dtype: int64

In [44]:
netflix1limpo = dfrelevante['Treinamento'].apply(cleanup)

print(netflix1limpo)

0      saiu da netflix q sdd de full house amo muitoo...
1      rt @karolcaipira gente vcs tem que entender um...
2      rt @karolcaipira gente vcs tem que entender um...
3      mano não tem mais nada p assistir na netflix q...
4      @jornaloglobo vcs assistiram o filme primeiram...
                             ...                        
291    rt @awaecharts anne with an e na netflix chile...
292    rt @baptistafairy eu tô muito sedenta pela seg...
293                               netflix já foi melhor 
297    @netflixbrasil netflix querida vê se consegue ...
298    rt @storyiaa never say never votou pra netflix...
Name: Treinamento, Length: 155, dtype: object


In [45]:
lista_relevantes = []
palavrasrelevantes = []
for tt in netflix1limpo:
    lista_relevantes.append(tt.split())
    
for a in lista_relevantes:
    for b in a:
        palavrasrelevantes.append(b)
        


In [46]:
serie_1 = pd.Series(palavrasrelevantes)
serie_1

0                    saiu
1                      da
2                 netflix
3                       q
4                     sdd
              ...        
2973                 meus
2974            lencinhos
2975                   de
2976                papel
2977    s//tco/edylx1xewt
Length: 2978, dtype: object

In [47]:
tabela_relevante = serie_1.value_counts()
tabela_relevante

netflix         152
a               107
de               70
e                62
que              57
               ... 
@rogwrspider      1
lembrei           1
kshsis            1
ohantoms          1
qui               1
Length: 1156, dtype: int64

## Laplace Smoothing

In [48]:
b = 1
v = 400000 #palavras na lingua port

## Calculando Palavras

In [49]:
total_palavras = tabela_naorelevante.sum() + tabela_relevante.sum()
total0 = len(tabela_naorelevante) + b*v
total1 = len(tabela_relevante) + b*v

In [50]:
probabilidade0 = tabela_naorelevante.sum() / total_palavras
probabilidade1 = tabela_relevante.sum() / total_palavras

## Ensinando Naive Bayes

In [51]:
def Classifica(tt, df):
    
    p_tt_dadonaorelevante = 1
    p_tt_dadorelevante = 1
    c = 0
    
    for palavra in tt:
        
        for a in palavra:
    
            if a in tabela_naorelevante:
                p_a_dadonaorelevante = (tabela_naorelevante[a] + b) / total0
                p_tt_dadonaorelevante *= p_a_dadonaorelevante
            else:
                p_a_dadonaorelevante = (b) / total0
                p_tt_dadonaorelevante *= p_a_dadonaorelevante
            
            if a in tabela_relevante:
                p_a_dadorelevante = (tabela_relevante[a] + b) / total1
                p_tt_dadorelevante *= p_a_dadorelevante
            else:
                p_a_dadorelevante = (b) / total1
                p_tt_dadorelevante *= p_a_dadorelevante
        
        p_tt_dadonaorelevante = p_tt_dadonaorelevante * probabilidade0
        p_tt_dadorelevante = p_tt_dadorelevante * probabilidade1
        
        
        if p_tt_dadonaorelevante > p_tt_dadorelevante:
            df.iloc[c, 2] = 0
        elif p_tt_dadorelevante > p_tt_dadonaorelevante:
            df.iloc[c, 2] = 1
            
        c += 1
        
        p_tt_dadonaorelevante = 1
        p_tt_dadorelevante = 1

### Aplicando a função

In [52]:
TOTAL = []
TOTALlimpo = train['Treinamento'].apply(cleanup)
for a in TOTALlimpo:
    TOTAL.append(a.split())

train['Cla'] = 'esther e lila'

Classifica(TOTAL, train)
train

,Treinamento,Categoria,Cla
0,saiu da netflix? q sdd de full house! amo muit...,1,1
1,rt @karolcaipira: gente vcs tem que entender u...,1,1
2,rt @karolcaipira: gente vcs tem que entender u...,1,1
3,"mano, não tem mais nada p assistir na netflix ...",1,1
4,"@jornaloglobo vcs assistiram o filme, primeira...",1,1
...,...,...,...
295,rt @craibeibe: inveja de você? kkkkkkkkkkkkkkk...,0,0
296,quero saber quando é q vou poder colocar as pi...,0,0
297,"@netflixbrasil netflix querida, vê se consegue...",1,1
298,rt @storyiaa: never say never votou pra netfli...,1,1


___
### Verificando a performance do Classificador

Agora você deve testar o seu classificador com a base de Testes.

In [53]:
testenetflixlimpo = test['Teste'].apply(cleanup)
tt_teste = []
for i in testenetflixlimpo:
    tt_teste.append(i.split())
    
test['Cla'] = 'esther e lila'

Classifica(tt_teste, test)
test

,Teste,Categoria,Cla
0,rt @blogdadireita: min. damares alves quer pro...,0,0
1,@rachwlgrwn no off eu fui até a terceira e se ...,1,1
2,eu quero julie e os fantasmas na netflix pra c...,1,0
3,nunca vou perdoar a netflix https://t.co/ssrbk...,1,0
4,seria demais querer que a @damaresalves e os e...,0,0
...,...,...,...
238,@damaresalves pra cima desses pedófilos da net...,1,0
239,to surtando com never say never na netflix mes...,1,1
240,greys anatomy lançou na globo play e eu só ten...,1,1
241,rt @larafurlanetto1: petição pra netflix adici...,1,1


In [54]:
train['Categoria'] = train['Categoria'].astype('category') 
train['Cla'] = train['Cla'].astype('category') 
train['Categoria'].cat.categories = ['Irrelevante', 'Relevante']
train['Cla'].cat.categories = ['Irrelevante', 'Relevante'] 



test['Categoria'] = test['Categoria'].astype('category') 
test['Cla'] = test['Cla'].astype('category') 
test['Categoria'].cat.categories = ['Irrelevante', 'Relevante'] 
test['Cla'].cat.categories = ['Irrelevante', 'Relevante']

In [55]:
pd.crosstab(train['Categoria'], train['Cla'], normalize = 'all')

Cla,Irrelevante,Relevante
Categoria,,
Irrelevante,0.46,0.023333
Relevante,0.01,0.506667


In [56]:
pd.crosstab(test['Categoria'], test['Cla'], normalize = 'all')

Cla,Irrelevante,Relevante
Categoria,,
Irrelevante,0.267490,0.168724
Relevante,0.209877,0.353909


___
### Concluindo

O classificador demonstrou um bom desempenho no planilha de treinamento (96% aproximadamente), já na planilha de testes ela apresentou uma qualidade menor que o esperado (60% aproximadamente). Dessa forma, concluimos que poderia ter sido feita mais filtrações e especificações na classsificação do excel. Isso pode ser comprovado nos dados abaixo:

In [57]:
print('Tabela cruzada Treinamento')
pd.crosstab(train['Categoria'], train['Cla'], normalize = 'all')


Tabela cruzada Treinamento


Cla,Irrelevante,Relevante
Categoria,,
Irrelevante,0.46,0.023333
Relevante,0.01,0.506667


In [58]:
print('Tabela cruzada Teste')
pd.crosstab(test['Categoria'], test['Cla'], normalize = 'all')

Tabela cruzada Teste


Cla,Irrelevante,Relevante
Categoria,,
Irrelevante,0.267490,0.168724
Relevante,0.209877,0.353909


___
## Aperfeiçoamento:

Os trabalhos vão evoluir em conceito dependendo da quantidade de itens avançados:

* Limpar: \n, :, ", ', (, ), etc SEM remover emojis
* Corrigir separação de espaços entre palavras e emojis ou entre emojis e emojis
* Propor outras limpezas e transformações que não afetem a qualidade da informação ou classificação
* Criar categorias intermediárias de relevância baseadas na probabilidade: ex.: muito relevante, relevante, neutro, irrelevante, muito irrelevante (3 categorias: C, mais categorias conta para B)
* Explicar por que não posso usar o próprio classificador para gerar mais amostras de treinamento
* Propor diferentes cenários para Naïve Bayes fora do contexto do projeto
* Sugerir e explicar melhorias reais com indicações concretas de como implementar (indicar como fazer e indicar material de pesquisa)
* Montar um dashboard que realiza análise de sentimento e visualiza estes dados

### Propor outras formas de limpeza

Para o nosso projeto, seria interessante as seguintes limpezas:

1- Pegar as derivações de palavras que tem o mesmo radical e contá-las como uma só palavra. Tal limpeza iria proporcionar um melhor resultado já que elas podem ter sido separadas em diferentes categorias, porém tem o mesmo significado.

2- Remover acentos de palavras. Principalmente no twitter, há muitas pessoas que escrevem palavras sem acento, mas também há aquelas que escrevem com, o que pode fazer com que essas sejam contadas como palavras distintas.

3- Pegar palavras e suas abreviações como por exemplo, a palavra "que" e sua abreviação "q". Tal mudança iria proporcionar uma melhoria na contabilização, ja que essas são consideradas palavras diferentes no classificador.

4- Uma pessoa somente estabelecer se o tweet é relevante ou não. Apesar de o grupo ter estabelecido um padrão, quando o assunto é ler frases e classificá-las de acordo com a sua relevância, pode ser que haja diferenças por ser algo subjetivo.

### Sugestão e melhorias reais

Acreditamos que para a melhoria de classificação, poderia ter sido analisada a frase como um todo e seus emojis caso apresente. Principalmente o emoji, é um bom indicador de satisfação e expressão de sentimentos, já ao analisar as frases como um todo pode-se analisar o contexto que elas estão (caso seja de ironia, sarcasmo, alegria e etc), para isso seria necessária o uso de outro sistema inclusive talvez um HTML. o material de pesquisa é de Stanford University e pode ser encontrado no link a seguir: https://web.stanford.edu/~jurafsky/slp3/4 

### Porque esse classificador não pode ser usado para gerar mais amostras de treinamento

O primeiro motivo de não poder usar o classificador, é porque demandaria outra planilha com o produto a ser analizado. Isso resultaria numa nova categorização da relevância e que possa ser mais ou menos complexo.
Bom, outro problema seria em relação a ter mais ou menos tweets, caso tenha menos, a classificação dos tweets tende a ser piorada por ter menos informações e classificações.
Por fim, ao realizar o Laplace Smoothing, nós estabelecemos uma alpha e um v que seria o número de palavras na língua portuguesa, então, caso os tweets sejam em outra língua, isso irá prejudicar os resultados.


### Propor diferentes cenários para Naive Bayes fora do contexto do projeto

O Naive Bayes poderia ser usado para classificar fake news por exemplo, dessa forma teria a possibilidade de reportar e adverttência do usuário. Tweets em relação algum outro produto, para caracterizar a satisfação ou desejo dos consumidores.

___
## Referências

[Naive Bayes and Text Classification](https://arxiv.org/pdf/1410.5329.pdf)  **Mais completo**

[A practical explanation of a Naive Bayes Classifier](https://monkeylearn.com/blog/practical-explanation-naive-bayes-classifier/) **Mais simples**

In [ ]:
#http://www.nltk.org/howto/portuguese_en.html 